In [ ]:
using Random
using LinearAlgebra
using Distributions
using Dates
using Profile
using BenchmarkTools

using PhDSE

In [ ]:
@info "" Base.current_project()

In [ ]:
include("_setup.jl")

In [ ]:
@info "Precompile"
_D, _d = 1000, 1000
_N = 200

const _Φ, _Q, _u, _H, _R, _v, _y, _μ₀, _Σ₀ = kalman_setup(D=_D, d=_d)
const _Rinv = inv(_R)

# Allocate memory
_fcache = EnKFCache(
    _D,
    _d,
    ensemble_size = _N,
    process_noise_dist = MvNormal(zeros(_D), _Q),
    observation_noise_dist = MvNormal(zeros(_d), _R),
)
_init_ensemble = rand(MvNormal(_μ₀, _Σ₀), _N)
copy!(_fcache.ensemble, _init_ensemble)
@info "Precompilation done"

In [ ]:
Ns = [10, 100, 500, 1000, 5000, 10000, 20000]

In [ ]:
Ds = [10, 100, 500, 1000, 5000, 10000, 20000]

In [ ]:
function bench(; D, d, N, benchmark_samples=1)
    Φ, Q, u, H, R, v, y, μ₀, Σ₀ = kalman_setup(D=D, d=d)
    Rinv = inv(R)
    
    # Allocate memory
    fcache = EnKFCache(
        D,
        d,
        ensemble_size = N,
        process_noise_dist = MvNormal(zeros(D), Q),
        observation_noise_dist = MvNormal(zeros(d), R),
    )
    init_ensemble = rand(MvNormal(μ₀, Σ₀), N)
    copy!(fcache.forecast_ensemble, init_ensemble)
    
    Profile.clear()
    bres_correct = @benchmark enkf_correct!($fcache, $H, $Rinv, $y, $v) samples=benchmark_samples
    
    return bres_correct
end

In [ ]:
in_ms(t::BenchmarkTools.Trial) = mean(t).time / 1e6

In [ ]:
bres_results_per_D = []
bres_results_per_N = []

In [ ]:
for D in Ds
    bres = bench(D=D, d=D, N=5)
    push!(bres_results_per_D, bres)
    println("D = $D ---> $(in_ms(bres))ms")
end

In [ ]:
for N in Ns
    bres = bench(D=5, d=5, N=N)
    push!(bres_results_per_N, bres)
    println("N = $N ---> $(in_ms(bres))ms")
end

In [ ]:
using Plots

In [ ]:
scatter(
    Ds, 
    map(in_ms, bres_results_per_D), 
    title="Runtime vs. state dimensions/ensemble size", 
    xlabel="dimensions/ensemble size", 
    ylabel="time [ms]", 
    label="D",
    legend=:bottomright,
)

In [ ]:
scatter!(
    Ns, 
    map(in_ms, bres_results_per_N), 
    label="N"
)